This notebook trains a KNN model, finds the optimal value of k, and reports f1 score and log loss.  

The notebook can use either the full set of training and test data from Kaggle, or on 80% of the training data and 20% held back development data.

In [1]:
import numpy as np
import pandas as pd
import zipfile
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [3]:
# Unzip data files into the "csv" subdirectory

# **IMPORTANT**  This will overwrite existing files in the "csv" folder in your local repo
# with the most recent data files from the data.zip file

# Unzip 80% training data
unzip_training_data = zipfile.ZipFile("data_subset.zip", "r")
unzip_training_data.extractall()
unzip_training_data.close()

# Unzip development and training data
unzip_test_data = zipfile.ZipFile("testing.zip", "r")
unzip_test_data.extractall()
unzip_test_data.close()

# Unzip full set of training data for creating predictions to submit to Kaggle
unzip_all_data = zipfile.ZipFile("data.zip", "r")
unzip_all_data.extractall()
unzip_all_data.close()

In [2]:
# Load these csv files into numpy arrays for testing on development data
train_data = np.loadtxt('csv/train_data.csv', delimiter=",")
train_labels = np.loadtxt('csv/train_labels.csv', dtype=str, delimiter=",")
dev_data = np.loadtxt('csv/dev_data.csv', delimiter=",")
dev_labels = np.loadtxt('csv/dev_labels.csv', dtype=str, delimiter=",")

In [7]:
# Load these csv files into numpy arrays for creating predictions to submit to Kaggle
train_data_all = np.loadtxt('csv/train_data_all.csv', delimiter=",")
train_labels_all = np.loadtxt('csv/train_labels_all.csv', dtype=str, delimiter=",")
test_data_all = np.loadtxt('csv/test_data_all.csv', delimiter=",")

In [3]:
# print shapes to compare before and after csv conversion
print("train_data shape is", train_data.shape)
print("train_labels shape is", train_labels.shape)
print("dev_data shape is", dev_data.shape)
print("dev_labels shape is", dev_labels.shape)

train_data shape is (702439, 58)
train_labels shape is (702439,)
dev_data shape is (175610, 58)
dev_labels shape is (175610,)


In [8]:
print("train_data_all shape is", train_data_all.shape)
print("train_labels_all shape is", train_labels_all.shape)
print("test_data_all shape is", test_data_all.shape)

train_data_all shape is (878049, 58)
train_labels_all shape is (878049,)
test_data_all shape is (884262, 58)


In [9]:
# IF there are additional changes to make to the data for this model
# that would be easier to do in pandas, uncomment and run this code. 
# This model works the same whether the data is in numpy or pandas, so presumably so do other models

#train_data = pd.DataFrame(train_data)
#train_labels = pd.DataFrame(train_labels)
#dev_data = pd.DataFrame(dev_data)
#dev_labels = pd.DataFrame(dev_labels)
#train_data_all = pd.DataFrame(train_data_all)
#train_labels_all = pd.DataFrame(train_labels_all)
#test_data_all = pd.DataFrame(test_data_all)


In [4]:
# Set up functions for training models and finding optimal value of k and reporting accuracy

def TrainKNN(data, labels, test_data, k=5):
    """This function takes in training data and labels, testing data,
    and can accept different values of k. 
    It trains a KNN model and returns the model and predicted probabilities.
    """
    KNN = KNeighborsClassifier(n_neighbors=k)
    KNN.fit(data, labels)
    pp = KNN.predict_proba(test_data)
    return KNN, pp

def find_k(data, labels, dev_data, dev_labels, k_values):
    """Find optimal value of k.  
    
    Note that this cannot be used on test data from Kaggle 
    because we do not have labels for that data.  This function is intended to only be used
    in the development stage with the development data.
    """
    for k in k_values:
        
        KNN, pp = TrainKNN(data, labels, dev_data, k)
        predictions = KNN.predict(dev_data)
        f1 = metrics.f1_score(dev_labels, predictions, average = "weighted")
        logloss = metrics.log_loss(dev_labels, pp)
        
        # Print F1 score and log loss for each value of k
        print("For k =", k, "the F1 score is", round(f1, 4), "and the Log Loss score is", round(logloss, 4))
    print("\n")
    

In [ ]:
# Find the optimal value of k using the 80% training data and the development data
# This takes too long to run with large values of k on the larger sized data set
k_values = [1, 7]
find_k(train_data, train_labels, dev_data, dev_labels, k_values)

In [ ]:
k_values = [15]
find_k(train_data, train_labels, dev_data, dev_labels, k_values)

In [ ]:
k_values = [25]
find_k(train_data, train_labels, dev_data, dev_labels, k_values)

In [4]:
# Can also train the model with single value of k with 80% training data and development data
k = 2
KNN, pp = TrainKNN(train_data, train_labels, dev_data, k)
logloss = metrics.log_loss(dev_labels, pp)
print(logloss)

24.0861679843


In [ ]:
# Before submitting to Kaggle, run the model on the full set of training data and test data
# using the optimal value for the model
k = use optimal value here
KNN, pp = TrainKNN(train_data_all, train_labels_all, test_data_all, k)

In [6]:
# Set up predictions for submission to Kaggle
headers = ["ARSON","ASSAULT","BAD CHECKS","BRIBERY","BURGLARY","DISORDERLY CONDUCT","DRIVING UNDER THE INFLUENCE",
           "DRUG/NARCOTIC","DRUNKENNESS","EMBEZZLEMENT","EXTORTION","FAMILY OFFENSES","FORGERY/COUNTERFEITING",
           "FRAUD","GAMBLING","KIDNAPPING","LARCENY/THEFT","LIQUOR LAWS","LOITERING","MISSING PERSON","NON-CRIMINAL",
           "OTHER OFFENSES","PORNOGRAPHY/OBSCENE MAT","PROSTITUTION","RECOVERED VEHICLE","ROBBERY","RUNAWAY",
           "SECONDARY CODES","SEX OFFENSES FORCIBLE","SEX OFFENSES NON FORCIBLE","STOLEN PROPERTY","SUICIDE",
           "SUSPICIOUS OCC","TREA","TRESPASS","VANDALISM","VEHICLE THEFT","WARRANTS","WEAPON LAWS"]
data = pd.DataFrame(data=pp, 
                    index=[x for x in range(len(test_data_all))], 
                    columns=headers)
data.columns.name ="Id"
print(data)

Create zipped csv file for Kaggle
#### Update the filename first in all lines of the following code
Add something unique after our names to avoid overwriting other submission files

In [ ]:
data.to_csv('Williams_Gascoigne_Vignola_.csv', index_label = "Id")

In [ ]:
zip_probs = zipfile.ZipFile("Williams_Gascoigne_Vignola_.zip", "w")
zip_probs.write("Williams_Gascoigne_Vignola_.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_probs.close()

### Results from previous datasets and/or model parameters

Results from dataset submitted as baseline (where the only feature engineering was Label Encoding into numeric features):  
(Baseline submission set k=1)

For k = 1 the F1 score is 0.2006  
For k = 1 the Log Loss score is 27.6013  
For k = 3 the F1 score is 0.1552  
For k = 3 the Log Loss score is 22.3636  
For k = 5 the F1 score is 0.154  
For k = 5 the Log Loss score is 19.0689  
For k = 7 the F1 score is 0.1533  
For k = 7 the Log Loss score is 16.6886  
For k = 9 the F1 score is 0.1526  
For k = 9 the Log Loss score is 14.8641  
For k = 11 the F1 score is 0.1515  
For k = 11 the Log Loss score is 13.3879  
For k = 15 the F1 score is 0.1492  
For k = 15 the Log Loss score is 11.2378  
